# Database Joins 3: Combining JIRA with JIRA

For this notebook, we start with the following research question. "Can we create data visualizations on top of the LESA, LPS, LPP, and BPR ticket metadata that lets us group together different tickets so that we can explore the times that tickets remain in each status based on those groupings?"

In order to investigate the answer to this question, we start with a much smaller sub-question that focuses on more recent data.

<b style="color:green">Can we create data visualizations on top of LPS, LPP, and BPR ticket metadata that let us group together different DXP tickets so that we can explore the times that tickets remain in each status based on those groupings?</b>

In order to answer this question, this notebook repeats the introduction of database joins with additional code samples.

At the end of this notebook, we will have a script that takes a sample of data from JIRA and enriches it with more data from JIRA, and the reader will have a further improved understanding of what goes into a join.

## Prerequisites

The following cell attempts to use `conda` and `pip` to install the libraries that are used by this notebook. If the output indicates that additional items were installed, you will need to restart the kernel after the installation completes before you can run the later cells in the notebook.

In [ ]:
!conda install matplotlib

## Notebook Imports

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function

from checklpp import *
import dateparser
import matplotlib
from matplotlib import pyplot as plt

## Fetch Related Tickets

There is a subset of tickets that may relate to DXP but won't be captured in our original query for LPP tickets, because they do not have an affected version field: SME requests. So, we'll need to fetch these separately.

To do that, we will use the `issueFunction` capability that has been provided by the [Adaptavist ScriptRunner](https://scriptrunner.adaptavist.com/latest/jira/jql-functions.html) JIRA plugin that has already been installed on [issues.liferay.com](https://issues.liferay.com/).

In [ ]:
def escape_jql_subquery(jql):
    return jql.replace('\\', '\\\\').replace('\"', '\\\"')

In [ ]:
sme_jql = """
    project = LPP and
    type in ("SME Request", "SME Request SubTask") and
    issueFunction in subtasksOf("%s")
""" % escape_jql_subquery(lpp_jql)

In [ ]:
if __name__ == '__main__':
    sme_issues = get_jira_issues(sme_jql, ['changelog'])
else:
    sme_issues = {}

We will make the assumption that the LPP ticket will link to any LPS tickets that are directly related to it, and we will fetch only those issues that have been directly referenced. Of course, LPP tickets also reference LPS tickets that aren't tied to the fix, so this is really a superset of what we're interested in.

In [ ]:
lps_jql = 'project = LPS and issueFunction in linkedIssuesOf("%s")' % escape_jql_subquery(lpp_jql)

In [ ]:
if __name__ == '__main__':
    lps_issues = get_jira_issues(lps_jql, ['changelog'])
else:
    lpp_issues = {}

BPR tickets won't be linked from the LPP ticket, but they will be linked from the LPS tickets that are referenced from the LPP tickets. So, we simply add one more level of nesting in order to fetch our BPR tickets.

In [ ]:
bpr_jql = 'project = BPR and issueFunction in linkedIssuesOf("%s")' % escape_jql_subquery(lps_jql)

In [ ]:
if __name__ == '__main__':
    bpr_issues = get_jira_issues(bpr_jql, ['changelog'])
else:
    bpr_issues = {}